<a href="https://colab.research.google.com/github/hilanistel/STCP_feature_selection/blob/GTCP/GTCP_MushroomsSmall_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')
#load users list and movies list
Mushrooms_data = pd.read_csv("gdrive/My Drive/תזה תואר שני/MushroomsSmall/Readings.csv",header=None)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
X=Mushrooms_data.drop(21, axis=1) # drop the reading class
Mushrooms_data=Mushrooms_data[Mushrooms_data.columns[len(Mushrooms_data.columns)-1]]
Mushrooms_readings = [tuple(row) for row in X.values]

In [26]:
TR = {}
for index_i, i in enumerate(Mushrooms_readings):
    TR.update({i:{}})
    for index_j, j in enumerate(Mushrooms_data):
      if index_j == index_i:
       TR[i] = Mushrooms_data[j]

In [ ]:
#TR = {(0,0,0,1):1, (0,1,1,1):1,(1,1,1,1):1,(1,0,2,1):1,(1,0,1,1):0,(0,1,0,0):0,(0,0,2,0):0}

In [28]:
ci = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

In [29]:
N = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

**CRA**

𝑆= {𝑖∈𝑁: 𝑟𝑖≠𝑞𝑖}

In [30]:
def S(r,q):
  s = []
  for index_i,i in enumerate(r):
      if r[index_i] != q[index_i]:
        s.append(index_i)
  return s

In [ ]:
S((1, 1, 1, 1),(1, 0, 1, 1))

[1]

If there is no 𝑆′∈ℂ s.t 𝑆′⊆𝑆

In [31]:
# function create to get all N subsets
def sub_lists (l): 
    base = []   
    lists = [base] 
    for i in range(len(l)): 
        orig = lists[:] 
        new = l[i] 
        for j in range(len(lists)): 
            lists[j] = lists[j] + [new] 
        lists = orig + lists   
    lists.remove([])  
    return lists

In [32]:
def S_in_C(S,C):
  subsets_S = sub_lists(S)
  for s in subsets_S:
    for c in C:
      if s == c:
        return 0 #True
  return 1 #False

In [33]:
C = [] # start with an empty collection of constraints
for i in TR:
  for j in TR:
    if TR[i] != TR[j]:
      s = S(i,j) # 𝑆= {𝑖∈𝑁: 𝑟𝑖≠𝑞𝑖}
      if S_in_C(s,C) == 1:
        for c in C:
          C1= C.copy()
          subsets_c = sub_lists(c)
          if s in subsets_c:
            C1.remove(c)
          C = C1
        C.append(s)
print(C)

[[1, 3, 4, 19], [1, 2, 3, 4], [4, 6, 9, 20], [4, 7], [0, 1, 3, 4], [1, 3, 4, 18], [1, 3, 4, 20], [1, 3, 4, 8]]


**CPLEX**

In [50]:
pip install cplex

     |████████████████████████████████| 31.0MB 150kB/s 


In [52]:

import cplex

# Create an instance of a linear problem to solve
problem = cplex.Cplex()


# We want to find a minimum of our objective function
problem.objective.set_sense(problem.objective.sense.minimize)

# The names of our variables (tests)
names = []
for i in N:
  names.append("x" + str(i))


# The obective function. More precisely, the coefficients of the objective
# function. Note that we are casting to floats.
objective = []
for i in ci:
  objective.append(float(i))

# Lower bounds. Since these are all zero, we could simply not pass them in as
# all zeroes is the default.
lower_bounds = []
for i in N:
  lower_bounds.append(float(0))

# Upper bounds. The default here would be cplex.infinity, or 1e+20.
upper_bounds = []
for i in N:
  upper_bounds.append(1000)

problem.variables.add(obj = objective,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = names)

# Constraints
constraint_names = []
for i in C:
  constraint_names.append("c" + str(i))

constraints = []
for i in C:
  var = []
  for j in i:
    var.append(names[j])
  ones = [1.0] *len(i)
  var_ones = [var, ones]
  constraints.append(var_ones)

rhs = [1.0] *len(C)
constraint_senses =["G"] *len(C)

# And add the constraints
problem.linear_constraints.add(lin_expr = constraints,
                               senses = constraint_senses,
                               rhs = rhs,
                               names = constraint_names)

# Solve the problem
problem.solve()

# And print the solutions
print(problem.solution.get_values())

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 11 columns.
Reduced LP has 8 rows, 10 columns, and 23 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)
Symmetry aggregator did 8 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =             1.000000

Dual crossover.
  Dual:  Fixing 4 variables.
        3 DMoves:  Infeasibility  1.66533454e-16  Objective  1.00000000e+00
        0 DMoves:  Infeasibility  3.33066907e-16  Objective  1.00000000e+00
  Dual:  Pushed 4, exchanged 0.
  Primal:  Fixed no variables.
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
